## 바른 형태소 분석기 활용 2(멀티라인 요청)
(이미 설치를 완료하신 분들은 아래 '샘플 텍스트 파일 다운로드' 부분부터 실행하면 됩니다.)

바른 설치
바른을 설치합니다.

바른의 리눅스 설치본을 다운로드 받습니다.

In [7]:
!curl -LJks -H "uname:$(uname -a)" https://bareun.ai/api/get -o bareun-linux.deb
!ls

bareun-linux.deb  sample_data


In [8]:
!uname -a

Linux e1408de574a0 5.10.147+ #1 SMP Sat Dec 10 16:00:40 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux


**바른**을 설치합니다.

In [9]:
!dpkg -i bareun-linux.deb

Selecting previously unselected package bareun.
(Reading database ... 122349 files and directories currently installed.)
Preparing to unpack bareun-linux.deb ...
Unpacking bareun (2.0.1) ...
Setting up bareun (2.0.1) ...
System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down
System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down
Created symlink /etc/systemd/system/multi-user.target.wants/bareun.service → /etc/systemd/system/bareun.service.


아래는 바른 모델이 GPU를 사용하기 위해서 다운받는 tensorflow 라이브러리입니다.

상단의 메뉴탭의 "런타임 > 런타임 유형 변경 > 하드웨어 가속기 > GPU 선택 > 저장"을 설정하시면 모델의 추론 속도가 더 빨라집니다. 단, 무료버전의 경우 coloab의 일일 사용 제한을 초과하지 않아야 합니다.

기본적으로 CPU를 사용해도 작동합니다.

In [10]:
!curl -O https://storage.googleapis.com/tensorflow/libtensorflow/libtensorflow-gpu-linux-x86_64-2.9.1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  392M  100  392M    0     0  17.9M      0  0:00:21  0:00:21 --:--:-- 19.2M


In [11]:
!tar -C /opt/bareun -xzf libtensorflow-gpu-linux-x86_64-2.9.1.tar.gz

## 환경 설정하기
아래 코드는 BAREUN_ROOT와 LD_LIBRARY_PATH를 설정하고, /opt/bareun/bin/bareun을 백그라운드로 실행하는 코드입니다.

In [12]:
%env BAREUN_ROOT="/opt/bareun"
%env LD_LIBRARY_PATH="/opt/bareun/lib"
!BAREUN_ROOT="/opt/bareun" LD_LIBRARY_PATH="/opt/bareun/lib" nohup /opt/bareun/bin/bareun&


env: BAREUN_ROOT="/opt/bareun"
env: LD_LIBRARY_PATH="/opt/bareun/lib"
nohup: appending output to 'nohup.out'


In [13]:
!ps -ef | grep bareun

root        5482       1 99 04:08 ?        00:00:05 /opt/bareun/bin/bareun
root        5519     358  0 04:08 ?        00:00:00 /bin/bash -c ps -ef | grep bareun
root        5521    5519  0 04:08 ?        00:00:00 grep bareun


API키 발급 바른 홈페이지(https://bareun.ai/) 에 회원가입해서 API키를 받습니다.

API키 복사
발급 받은 API키를 -reg 인자 다음에 입력하여 등록합니다.

In [ ]:
!BAREUN_ROOT="/opt/bareun" LD_LIBRARY_PATH="/opt/bareun/lib" /opt/bareun/bin/bareun -reg YOUR-API-KEY

## 바른 사용하기
**PIP 패키지 설치**


In [17]:
!pip install -U bareunpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
import sys
import bareunpy as brn
import google.protobuf.text_format as tf
API_KEY = "YOUR-API-KEY"
tagger = brn.Tagger(API_KEY, "localhost", 5656)

## 샘플 텍스트 파일 다운로드

* 15만 문장으로 구성된 샘플 텍스트 파일입니다.

In [ ]:
!pip install -U gdown

In [ ]:
!gdown  https://drive.google.com/file/d/1ajJlbc-i_1nCA92tx7lUuCwft4Qkwbfi/view?usp=share_link -O test_sentences.txt --fuzzy

In [ ]:
!pip install tqdm

## 대용량 코퍼스 처리 

* 바른 형태소 분석기를 통해서 많은 문장을 처리하는데 효율적인 방법으로 한 번에 하나의 문장을 요청하는 것이 아닌 많은 문장을 합쳐서 한 번에 요청하는 방법이 있습니다.
* 이렇게 요청을 해도 거의 동일한 결과를 얻을 수 있습니다.
* '\n' 줄바꿈 문자로 구분된 txt 파일을 불러와서 여러 문장을 하나의 문장으로 합쳐서 형태소 분석을 요청하는 것입니다.
* 이렇게 하면 많은 양의 텍스트도 빠르게 처리할 수 있습니다. 
* 아래 코드를 실행하면 yaml 파일로 문장과 해당 문장의 형태소 분석 결과가 저장됩니다. GPU를 사용한다면 47초 정도 소요되디만, 1문장씩 요청한다면 

In [ ]:
import yaml
import time
from tqdm import tqdm

sent_dict = []
time1 = time.time()

with open("test_sentences.txt", 'r', encoding='utf-8') as f:
  lines = f.readlines()           
  len_lines = len(lines)
  num_itters = len_lines//500 + 1 if len_lines%500 != 0 else len_lines//500
  tt = []    
  for i in range(num_itters):
      tt.append("".join(lines[500*i:500*(i+1)]))
  for text in tqdm(tt):
      res = tagger.tags([text]).msg()
      len_sent = len(res.sentences)
      for idx in range(0, len_sent):
          key = res.sentences[idx].text.content
          sentence = {key: []}      
          for tok in res.sentences[idx].tokens:
              sentence[key].append({
                          "token": tok.text.content,
                          "tag": tok.tagged
                      })
          sent_dict.append(sentence)

elapsed_call_time = time.time() - time1
print(f'elapsed_call_time: {elapsed_call_time:.3f}')

with open(f'test_sentences.yaml', 'w', encoding='utf-8') as outfile:
    yaml.dump(sent_dict, outfile, default_flow_style=False, allow_unicode=True, sort_keys=False)
print('saved output -> test_sentences.yaml')

* GPU 사용, 다중 문장 요청, 47초 소요, 약 19배 향상
```
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:47<00:00,  7.69it/s]
elapsed_call_time: 47.707
saved output -> test_sentences.yaml
```

* GPU 사용, 한 개의 문장씩 요청한 경우, 961초 소요 
```
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150000/150000 [16:01<00:00, 156.04it/s]
elapsed_call_time: 961.348
saved output -> test_sentences.yaml
```

